In [1]:
import requests
from bs4 import BeautifulSoup
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv

In [2]:
# Set up Spotipy credentials
client_id="6c3bb7a110824328ac2ce9e7412b306d"
client_secret="e877cedaf95142a8813bdd95c72e6cc8"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Warning: The following code does not use batch processing, so it may take a while to complete (around 30 minutes). If necessary to re-run this code, it is advised to modify it so that it utilizes Spotify's batch processing features (which can process around 100 songs at a time) for faster runtime.

In [3]:
# URL of the website
url = 'https://kworb.net/pop/cumulative.html'

# Make a GET request to the URL and get the response
response = requests.get(url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table element on the page by its ID
table = soup.find('table')

# Find all rows in the table body
rows = table.tbody.find_all('tr')

In [4]:
# Create a new CSV file in write mode
with open('kworb_popularity.csv', mode='w', newline='') as f:
    # Create a CSV writer object
    csv_writer = csv.writer(f)

    # Loop through each row and extract the data from the cells
    for row in rows:
        cells = row.find_all('td')
        
        # Extract the text content of each cell and add it to a row
        row_data = []
        for cell in cells:
            row_data.append(cell.text.strip())

        # Search for the song on Spotify and extract the first result
        song_name = row_data[1]
        results = sp.search(q=song_name, type='track')

        # If results exist, get audio features of track and add them to row_data
        if results['tracks']['items']:
            track_id = results['tracks']['items'][0]['id']
            audio_features = sp.audio_features([track_id])
            row_data.extend(audio_features)

        # Write the row data to the CSV file (if no Spotify track is found, empty columns will be filled with None)
        csv_writer.writerow(row_data)

KeyboardInterrupt: 